# LambdaFace

In [ ]:
from distutils.command.clean import clean
from email.mime import image
import json
import os
import logging
import boto3
import urllib
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor
import base64



def lambda_handler():
    # TODO implement

    # logger = logging.getLogger()
    # logger.setLevel(logging.INFO)

    # bucket = event['Records'][0]['s3']['bucket']['name']
    # key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')

    bucket='analitica-rekognition-bucket'
    key='caras.jpg'
   
    print('Llamada de crop')

    dictionary, num, image= detect_faces(bucket,key)

    print('\n Resultado final: \n')
    print(dictionary)
    print('número de personas: ' + '{0:.0f}'.format(num) + '\n')

    print('Llamada de crop')
    listaimg=cropFace(image,dictionary,num,key)

    for image in listaimg:

        image_data = Image.open(io.BytesIO(image))
        var = 'JPEG'
        # image_data.save(r'C:\Users\user\Desktop',var)
        image_data.show()

        #Se busca si la persona esta en una collection
        #Se recibe un booleano donde true indica que si pertence y false que no pertenece
        faceids=search_faces(image)

        #Se actualiza atributo (Status) en la base de datos en DynamoDB
        #de acuerdo al resultado de la función search_faces para cada una de las coincidencias en la collection
        for faceid in faceids:
            updateItemDB(faceid)



   

def detect_faces(bucket,key):

    client=boto3.client('rekognition', 'us-east-1')


    print(bucket)
    print(key)
    
    response = client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': key}},
        Attributes=['ALL'])

    print('Detected faces for ' + key)   

    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
            + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

        # print('Here are the other attributes:')
        # print(json.dumps(faceDetail, indent=4, sort_keys=True))

        # Access predictions for individual face details and print them
        print("Gender: " + str(faceDetail['Gender']))
        # print("Smile: " + str(faceDetail['Smile']))
        # print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
        # print("Emotions: " + str(faceDetail['Emotions'][0]))


    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')
    s3_object = s3_connection.Object(bucket,key)
    s3_response = s3_object.get()

    

    stream = io.BytesIO(s3_response['Body'].read())

    print('Hasta aquí se ejecura sin PIL')
    image=Image.open(stream)

    imgWidth, imgHeight = image.size  

    print('Se va a imprimir imagen')

    # draw = ImageDraw.Draw(image)  
                
    dict={}


    count=1

    # calculate and display bounding boxes for each detected face       
    # print('Bounding boxes for ' + key)    
    for faceDetail in response['FaceDetails']:
    

        
        box = faceDetail['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']

        dict['cara'+'{0:.0f}'.format(count)]=[left,top,left+width,top+height]
                
        count=count+1


        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Face Width: ' + "{0:.0f}".format(width))
        print('Face Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)

        )
        # draw.line(points, fill='#00d400', width=2)

    print('número de caras')
    numCaras=len(response['FaceDetails'])


    # image.show()

    return dict, numCaras, image


def cropFace(image,dict,numCaras,key):

    client=boto3.client('s3')

    key=key[0:len(key)-4]

    listaimg=[]

    for i in range(1,numCaras+1):
        
        nombre=key+"{0:.0f}".format(i)

        dimensiones=dict['cara'+"{0:.0f}".format(i)]

        dim=(int(dimensiones[0]),int(dimensiones[1]),int(dimensiones[2]),int(dimensiones[3]))
        imagecrop=image.crop(dim)


        img_byte_arr = io.BytesIO()


        imagecrop.save(img_byte_arr, format="JPEG")

        # img_str = base64.b64encode(img_byte_arr.getvalue())

        # imagecrop.save(img_byte_arr, format='JPEG')

        image_file_size = img_byte_arr.tell()

        print('Tamaño (bytes)')
        print(image_file_size)

        img_byte_arr = img_byte_arr.getvalue()

        listaimg.append(img_byte_arr)

    #     print(nombre+'.jpeg')

    #     # client.upload_fileobj(img_str.read(), 'prueba-rekognition-analitica', nombre+'.jpeg')

    #     client.put_object(
    # Body=img_byte_arr,
    # Bucket='prueba-rekognition-analitica',
    # Key=nombre+'.jpeg',

# )

    print(len(listaimg))

    return listaimg

#Se define función para buscar caras en una imagen y relacionar con una collection
#Recibe el nombre del bucket y de la imagen a analizar 
#Retorna lista con los FaceId de las coincidencias en la colección
def search_faces(image):

    #Cliente representando servicio de rekognition
    client=boto3.client('rekognition', 'us-east-1')


    collectionId = 'CollectionAnalitica' #Nombre de la colección
    threshold = 80 #Umbral para similaridad entre caras
    maxFaces = 100 #Número máximo de caras que quiere reconocer de la colección
    
    #Función del SDK (boto3) de python para buscar coincidencia con caras de una colección


    response=client.search_faces_by_image(CollectionId=collectionId,
                                    Image={'Bytes': image},
                                    FaceMatchThreshold=threshold,
                                     MaxFaces=maxFaces)


    faceMatches = response['FaceMatches']

    #Lista con el FaceId de la cara de coincidencia en la colección
    listface=[]


    for match in faceMatches:
        print('FaceId:' + match['Face']['FaceId'])
        print('ImageId:' + match['Face']['ImageId'])
        print('Similarity: ' + "{:.2f}".format(match['Similarity']) + "%")
        print('Confidence: ' + str(match['Face']['Confidence']))

        #Si la similaridad entre coincidencia es mayor a 80% se agrega el FaceId a la lista listface
        if match['Similarity'] > 80.0:
            listface.append( match['Face']['FaceId'])

    return listface


#Se define función para actualizar un item de la base de datos de dynamodb
#Recibe el FaceId del item a actualizar
def updateItemDB(FaceId):

    #Cliente representando servicio dynamodb
    client = boto3.client('dynamodb')

    try:

        #Se actualiza la base de datos cambiando el atributo status con el valor booleano True
        response = client.update_item(
            TableName='rekognition-machine-analitic',
            Key={
                'FaceID': {
                    'S': FaceId
                }
            },
            AttributeUpdates={
                'Status': {
                    'Value': {
                        'S': 'OK'
                    },
                    'Action': 'PUT'
                }
            },


        )   

        print('Actualizó DB')

    except Exception as msg:

        print(f"Oops, no se pudo actualizar el item: {msg}")

lambda_handler()

SNS

In [10]:
import boto3
import json

notification = "Here is the SNS notification for Lambda function tutorial."
client = boto3.client('sns')

response = client.publish (
    TargetArn = "arn:aws:sns:us-east-1:533886999211:SNSPrueba",
    Message = json.dumps({'default': notification}),
    MessageStructure = 'json'
)



In [11]:
import boto3
import json

client = boto3.client('sns')

snsArn = 'arn:aws:sns:us-east-1:533886999211:FaceSNS'
message = "This is a test notification."

response = client.publish(
    TopicArn = snsArn,
    Message = message ,
    Subject='Hello'
)

In [ ]:
response = client.get_item(
    TableName='string',
    Key={
        'string': {
            'S': 'string',
            'N': 'string',
            'B': b'bytes',
            'SS': [
                'string',
            ],
            'NS': [
                'string',
            ],
            'BS': [
                b'bytes',
            ],
            'M': {
                'string': {'... recursive ...'}
            },
            'L': [
                {'... recursive ...'},
            ],
            'NULL': True|False,
            'BOOL': True|False
        }
    },
    AttributesToGet=[
        'string',
    ],
    ConsistentRead=True|False,
    ReturnConsumedCapacity='INDEXES'|'TOTAL'|'NONE',
    ProjectionExpression='string',
    ExpressionAttributeNames={
        'string': 'string'
    }
)

In [2]:
import cv2
cap = cv2.VideoCapture("rtsp://192.168.1.16:554/live1s1.sdp",cv2.CAP_FFMPEG)

 #Tamaño de la imagen transmitida
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH )
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(width)
print(height)

#Se ajusta tamaño de la recepción de acuerdo al tamaño de
#la imagen del video transmitido.
cap.set(3, width)
cap.set(4, height)

if not cap.isOpened():
    print('Cannot open RTSP stream')
    exit(-1)

while True:
    _, frame = cap.read()
    cv2.namedWindow("RTSP stream", cv2.WINDOW_NORMAL)
    cv2.imshow('RTSP stream', frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

800.0
1080.0


In [10]:
import base64
with open("/home/analitica2/analitica2/20220921_155031M.jpg", "rb") as img_file:
    if img_file.read():
        print("leyo bien")
        print((base64.b64encode(img_file.read()).decode('utf-8')))

leyo bien



In [9]:
import pika
import json
#método para públicar
def publish(message, queue, mqHost):

    #Se establece conexión con el servidor
    connection = pika.BlockingConnection(pika.ConnectionParameters(mqHost))

    #Se comienza el canal de comunicación
    channel = connection.channel()

    #Se establece la cola
    channel.queue_declare(queue, passive=False, durable=False, exclusive=False, auto_delete=False, arguments=None)

    #Se envía mensaje
    channel.basic_publish("", queue, bytes(message, 'utf-8'), properties=None, mandatory=False)
    print(" [x] Sent %r" % message)

    #Se cierra la conexión
    connection.close()

data = {}
            
data["path"] = "/home/analitica2/Documentos/ftp/20220922_073004M.jpg"
mqHost = "localhost"   

#Se envía mensaje a la cola del servicio de mensajería
publish(json.dumps(data), "captured-image-queue", mqHost)

print("Se envio imagen")
        








 [x] Sent '{"path": "/home/analitica2/Documentos/ftp/20220922_073004M.jpg"}'
Se envio imagen


In [ ]:
{"Records":[{"eventVersion":"2.1","eventSource":"aws:s3","awsRegion":"us-east-1","eventTime":"2022-10-10T11:46:51.538Z","eventName":"ObjectCreated:Put","userIdentity":{"principalId":"A2RRAKSSN6SOYW"},"requestParameters":{"sourceIPAddress":"179.32.28.230"},"responseElements":{"x-amz-request-id":"PPF94S9K5PY231C0","x-amz-id-2":"+hUloFhsu8lbc8s8pKEHNzI9MrBF/dWuQnO5YIaNSYd50Bc+aRKYQVsPpXPzOeFyH1hdWLVkBMw98qvW+koDwIrVYPE9lnrwwEZK8IIWsPg="},"s3":{"s3SchemaVersion":"1.0","configurationId":"kinesis_captures","bucket":{"name":"bucket-gstreamer","ownerIdentity":{"principalId":"A2RRAKSSN6SOYW"},"arn":"arn:aws:s3:::bucket-gstreamer"},"object":{"key":"20221078569343373.jpg","size":265525,"eTag":"f9a6d0e79122ee2c9ad59032db67e1c1","sequencer":"006344062B6C9901B9"}}}]}

## CREATE COLLECTION

In [1]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def create_collection(collection_id):

    client=boto3.client('rekognition')

    #Create a collection
    print('Creating collection:' + collection_id)
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))
    print('Done...')
    
def main():
    collection_id='CollectionKinesis'
    create_collection(collection_id)

if __name__ == "__main__":
    main()    


Creating collection:CollectionKinesis
Collection ARN: aws:rekognition:us-east-1:853702706419:collection/CollectionKinesis
Status code: 200
Done...


In [9]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def add_faces_to_collection(bucket,photo,collection_id):


    
    client=boto3.client('rekognition')

    response=client.index_faces(CollectionId=collection_id,
                                Image={'S3Object':{'Bucket':bucket,'Name':photo}},
                                ExternalImageId=photo,
                                MaxFaces=1,
                                QualityFilter="AUTO",
                                DetectionAttributes=['ALL'])

    print ('Results for ' + photo) 	
    print('Faces indexed:')						
    for faceRecord in response['FaceRecords']:
         print('  Face ID: ' + faceRecord['Face']['FaceId'])
         print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))

    print('Faces not indexed:')
    for unindexedFace in response['UnindexedFaces']:
        print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
        print(' Reasons:')
        for reason in unindexedFace['Reasons']:
            print('   ' + reason)
    return len(response['FaceRecords'])

def main():
    bucket='bucket-collection-kinesis'
    collection_id='CollectionKinesis'
    photo='selene1.jpg'
    
    
    indexed_faces_count=add_faces_to_collection(bucket, photo, collection_id)
    print("Faces indexed count: " + str(indexed_faces_count))


if __name__ == "__main__":
    main()

Results for selene1.jpg
Faces indexed:
  Face ID: 2e2debd8-c79d-4d2f-925b-b40311a27ee4
  Location: {'Width': 0.22858935594558716, 'Height': 0.24755693972110748, 'Left': 0.3399488031864166, 'Top': 0.48553362488746643}
Faces not indexed:
Faces indexed count: 1


### List faces in a collection

In [10]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def list_faces_in_collection(collection_id):


    maxResults=2
    faces_count=0
    tokens=True

    client=boto3.client('rekognition')
    response=client.list_faces(CollectionId=collection_id,
                               MaxResults=maxResults)

    print('Faces in collection ' + collection_id)

 
    while tokens:

        faces=response['Faces']

        for face in faces:
            print (face)
            faces_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']
            response=client.list_faces(CollectionId=collection_id,
                                       NextToken=nextToken,MaxResults=maxResults)
        else:
            tokens=False
    return faces_count   
def main():

    collection_id='CollectionKinesis'

    faces_count=list_faces_in_collection(collection_id)
    print("faces count: " + str(faces_count))
if __name__ == "__main__":
    main()

Faces in collection CollectionKinesis
{'FaceId': '014c8443-2594-46d3-bd61-659a27a7790c', 'BoundingBox': {'Width': 0.23402300477027893, 'Height': 0.23601199686527252, 'Left': 0.2742139995098114, 'Top': 0.45376500487327576}, 'ImageId': 'a6eace65-70cc-326c-b284-17b37b91c661', 'ExternalImageId': 'selene.jpg', 'Confidence': 99.99949645996094, 'IndexFacesModelVersion': '6.0'}
{'FaceId': '2e2debd8-c79d-4d2f-925b-b40311a27ee4', 'BoundingBox': {'Width': 0.2285889983177185, 'Height': 0.24755699932575226, 'Left': 0.33994901180267334, 'Top': 0.48553401231765747}, 'ImageId': '1af1596f-e68d-30d9-b8fb-56e6fa4ca2b0', 'ExternalImageId': 'selene1.jpg', 'Confidence': 99.99979400634766, 'IndexFacesModelVersion': '6.0'}
{'FaceId': '6e9c1b59-0195-48fc-98bc-95fc7d2e3f16', 'BoundingBox': {'Width': 0.5574470162391663, 'Height': 0.6843469738960266, 'Left': 0.15246300399303436, 'Top': 0.09815199673175812}, 'ImageId': 'ef41e85a-4fee-3394-960e-f241079cf746', 'ExternalImageId': 'Emanuel2.jpg', 'Confidence': 99.9984